## IMPORT PACKAGE

In [39]:
import os
import glob
import random
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings
from langchain.schema import HumanMessage
from langchain.chains import RetrievalQA
from dotenv import load_dotenv

In [40]:
load_dotenv()
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")

## DATA

In [32]:
# Load all PDFs from the specified folder
pdf_folder_path = "../Data/"
all_pdf_paths = glob.glob(os.path.join(pdf_folder_path, "*.pdf"))

In [33]:
# Load each PDF document and split text
documents = []
for pdf_path in all_pdf_paths:
    loader = PyPDFLoader(pdf_path)
    pdf_docs = loader.load()
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
    documents.extend(text_splitter.split_documents(pdf_docs))

print(f"Total loaded document chunks: {len(documents)}")

Total loaded document chunks: 1484


## CHUNKING

In [34]:
# Display a random chunk for verification
random_chunk = random.choice(documents)
print("Randomly selected document chunk:")
print(random_chunk)

Randomly selected document chunk:
page_content='Penelitian pada pasien PPOK eksaserbasi rawat jalan 
menunjukkan hubungan antara purulensi sputum 
dengan terdapatnya bakteri. Penelitian PPOK eksaserbasi 
menggunakan ventilasi mekanis yang tidak diberikan 
antibiotik akan meningkatkan mortalitas dan 
meningkatnya angka kejadian pneumonia nosokomial. 
Antibiotik diberikan pada : 
a) Pasien PPOK eksaserbasi dengan semua gejala 
kardinal (sesak nafas yang bertambah,' metadata={'source': '../Data\\COPD-Kemenkes.pdf', 'page': 62}


## EMBEDDING

In [35]:
# Set up embeddings and LLM with Google Gemini API
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001", google_api_key=GEMINI_API_KEY)
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash", google_api_key=GEMINI_API_KEY)

In [36]:
# Create FAISS vector database from documents
vector_db = FAISS.from_documents(documents, embeddings)
retriever = vector_db.as_retriever(search_type="similarity", search_kwargs={"k": 5})

## RAG

In [41]:
# Function to interact with user and generate RAG answers
def interact_with_user():
    dataset = []  # List to store question and answer pairs for evaluation
    
    print("Hello! I'm here to answer your questions based on the documents.")
    while True:
        # Ask the user for a question
        question = input("Masukkan pertanyaan anda (or type 'exit' to quit): ")
        
        # Exit condition
        if question.lower() == 'exit':
            print("Goodbye!")
            break
        
        # Retrieve relevant documents for the question
        retrieved_docs = retriever.get_relevant_documents(question)
        
        # Prepare the prompt for Gemini LLM
        prompt = f"Berdasrkan informasi pada dokumen tentang penyakit PPOK, jawbalah pertanyaan berikut:\n\n"
        prompt += "\n".join([doc.page_content for doc in retrieved_docs])  # Add content from retrieved docs
        prompt += f"\n\nQuestion: {question}\nAnswer: "
        
        # Generate answer from Gemini LLM
        response = llm([HumanMessage(content=prompt)])
        answer = response.content  # Extract the generated answer
        
        # Display the answer to the user
        print(f"Answer: {answer}\n")
        
        # Save the question and answer pair for future evaluation
        dataset.append({"question": question, "generated_answer": answer})
        
    return dataset

In [42]:
# Start interaction with user
generated_dataset = interact_with_user()

Hello! I'm here to answer your questions based on the documents.
Answer: PPOK adalah penyakit paru kronik yang ditandai oleh hambatan aliran udara di saluran nafas yang tidak sepenuhnya reversible. 


Answer: Berdasarkan informasi yang Anda berikan, berikut terminologi yang terkait dengan PPOK:

**Gejala:**

* **Sesak:** 
    * Progresif (sesak bertambah berat seiring berjalannya waktu)
    * Bertambah berat dengan aktivitas
    * Menetap sepanjang hari
    * Dijelaskan oleh bahasa pasien sebagai "Perlu usaha untuk bernafas,"
    * Berat, sukar bernafas, terengah-engah
* **Batuk kronik:**
    * Hilang timbul dan mungkin tidak berdahak
    * Berdahak (Setiap batuk kronik berdahak dapat mengindikasikan PPOK)

**Faktor Risiko:**

* **Asap rokok**
* **Polusi udara:** 
    * Dalam ruangan
    * Luar ruangan
* **Pajanan zat di tempat kerja**
* **Genetik**
* **Usia dan jenis kelamin**
* **Tumbuh kembang paru**
* **Sosial ekonomi**
* **Infeksi paru berulang**
* **Asma/hiperreaktivitas bronkus*

In [43]:
# Save the dataset (question, generated_answer) for future evaluation
import json
with open("./rag_evaluation_dataset.json", "w") as f:
    json.dump(generated_dataset, f, indent=4)

print("Dataset saved for future evaluation.")

Dataset saved for future evaluation.


## EVALUATION

In [75]:
import json

# Data yang ada
rag_dataset = [
    {
        "question": "apa itu PPOK",
        "generated_answer": "PPOK adalah penyakit paru kronik yang ditandai oleh hambatan aliran udara di saluran nafas yang tidak sepenuhnya reversible. \n"
    },
    {
        "question": "Apa saja terminologi pada PPOK?",
        "generated_answer": "Berdasarkan informasi yang Anda berikan, berikut terminologi yang terkait dengan PPOK:\n\n**Gejala:**\n\n* **Sesak:** \n    * Progresif (sesak bertambah berat seiring berjalannya waktu)\n    * Bertambah berat dengan aktivitas\n    * Menetap sepanjang hari\n    * Dijelaskan oleh bahasa pasien sebagai \"Perlu usaha untuk bernafas,\"\n    * Berat, sukar bernafas, terengah-engah\n* **Batuk kronik:**\n    * Hilang timbul dan mungkin tidak berdahak\n    * Berdahak (Setiap batuk kronik berdahak dapat mengindikasikan PPOK)\n\n**Faktor Risiko:**\n\n* **Asap rokok**\n* **Polusi udara:** \n    * Dalam ruangan\n    * Luar ruangan\n* **Pajanan zat di tempat kerja**\n* **Genetik**\n* **Usia dan jenis kelamin**\n* **Tumbuh kembang paru**\n* **Sosial ekonomi**\n* **Infeksi paru berulang**\n* **Asma/hiperreaktivitas bronkus**\n* **Bronkitis kronik**\n\n**Lainnya:**\n\n* **Penyakit Paru Obstruktif Kronik (PPOK)**\n* **Bronkitis obstruktif**\n* **Emfisema**\n* **Asma**\n* **Resistensi**\n\n**Istilah lain yang mungkin terkait:**\n\n* **Pedoman Tata Laksana PPOK**\n* **Diagnosis dini**\n* **Tata laksana**\n* **Rehabilitasi**\n\nSemoga informasi ini membantu! \n"
    },
    {
        "question": "1990 PPOK  menempati urutan kematian keberapa dunia?",
        "generated_answer": "Berdasarkan informasi yang diberikan, pada tahun 1990 PPOK menempati **urutan ke-6** sebagai penyebab utama kematian di dunia. \n"
    },
    {
        "question": "pada tahun 1990 PPOK  menempati urutan kmetain keberapa di dunia",
        "generated_answer": "Berdasarkan informasi pada dokumen, pada tahun 1990 PPOK menempati urutan **keenam** sebagai penyebab utama kematian di dunia. \n"
    },
    {
        "question": " Biaya pengobatan PPOK di Amerika Serikat pada tahun 2010 berapa?",
        "generated_answer": "Biaya pengobatan PPOK di Amerika Serikat pada tahun 2010 sekitar **50 miliar Dollar AS**. \n"
    },
    {
        "question": "Tahun 2006 berapa kasus PPOK di Indoensia?",
        "generated_answer": "Teks tersebut tidak menyebutkan jumlah kasus PPOK di Indonesia pada tahun 2006. Teks hanya menyebutkan prevalensi PPOK di Indonesia pada tahun 2012 sebesar 4,5%. \n"
    }
]

In [76]:
# Jawaban yang diharapkan
expected_answers = [
    "PPOK adalah penyakit paru kronik yang ditandai oleh hambatan aliran udara di saluran nafas yang tidak sepenuhnya reversible.",
    "Terminologi pPOK meliputi PPOK dini, PPOK ringan, PPOK pada usia muda, Pra-PPOK, dan PRISm",
    "Pada tahun 1990, PPOK menempati urutan ke-6 sebagai penyebab utama kematian di dunia.",
    "Pada tahun 1990, PPOK menempati urutan keenam sebagai penyebab utama kematian di dunia.",
    "Biaya pengobatan PPOK di Amerika Serikat pada tahun 2010 sekitar 50 miliar Dollar AS.",
    "Teks tidak menyebutkan jumlah kasus PPOK di Indonesia pada tahun 2006, hanya menyebutkan prevalensi pada tahun 2012."
]

# Gabungkan data menjadi satu
combined_data = []
for idx, item in enumerate(rag_dataset):
    combined_data.append({
        "question": item["question"],
        "generated_answer": item["generated_answer"],
        "expected_answer": expected_answers[idx]
    })

# Simpan dataset gabungan dalam format JSON
with open("./rag_combined_evaluation_dataset.json", "w") as f:
    json.dump(combined_data, f, indent=4)

print("Dataset gabungan telah disimpan sebagai rag_combined_evaluation_dataset.json.")

Dataset gabungan telah disimpan sebagai rag_combined_evaluation_dataset.json.


In [77]:
from sklearn.metrics import precision_score, recall_score

In [78]:
def evaluate_answer_relevancy(generated_answer, expected_answer):
    # Evaluasi relevansi: menggunakan persentase kecocokan kata-kata sebagai contoh sederhana
    generated_set = set(generated_answer.lower().split())
    expected_set = set(expected_answer.lower().split())
    intersection = len(generated_set.intersection(expected_set))
    total = len(expected_set)
    return intersection / total if total > 0 else 0

def evaluate_faithfulness(generated_answer, expected_answer):
    # Evaluasi kesesuaian: menggunakan persentase kecocokan kalimat yang lebih ketat
    return 1.0 if generated_answer.lower() == expected_answer.lower() else 0.0

def evaluate_contextual_recall(generated_answer, expected_answer):
    # Recall: Berapa banyak informasi yang relevan ditemukan dalam jawaban
    return evaluate_answer_relevancy(generated_answer, expected_answer)

def evaluate_contextual_precision(generated_answer, expected_answer):
    # Precision: Seberapa banyak informasi yang dimasukkan dalam jawaban adalah relevan
    return evaluate_answer_relevancy(generated_answer, expected_answer)

def evaluate_contextual_relevancy(generated_answer, expected_answer):
    # Kombinasi relevansi dan recall
    return (evaluate_contextual_recall(generated_answer, expected_answer) + evaluate_contextual_precision(generated_answer, expected_answer)) / 2

# Evaluasi dataset gabungan
evaluations = []
for data in combined_data:
    generated_answer = data["generated_answer"]
    expected_answer = data["expected_answer"]
    
    relevancy = evaluate_answer_relevancy(generated_answer, expected_answer)
    faithfulness = evaluate_faithfulness(generated_answer, expected_answer)
    recall = evaluate_contextual_recall(generated_answer, expected_answer)
    precision = evaluate_contextual_precision(generated_answer, expected_answer)
    relevancy_context = evaluate_contextual_relevancy(generated_answer, expected_answer)

    evaluations.append({
        "question": data["question"],
        "relevancy": relevancy,
        "faithfulness": faithfulness,
        "recall": recall,
        "precision": precision,
        "contextual_relevancy": relevancy_context
    })

# Simpan hasil evaluasi
with open("./rag_evaluation_results.json", "w") as f:
    json.dump(evaluations, f, indent=4)

print("Evaluasi RAG telah disimpan dalam rag_evaluation_results.json.")

Evaluasi RAG telah disimpan dalam rag_evaluation_results.json.
